<a href="https://colab.research.google.com/github/caua-sathler/NEURAL-NETWORKS/blob/main/MLP-PSO-SGD/MLP_PSO_Adam_Ionosphere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data"
column_names = [f"feature_{i}" for i in range(34)] + ["label"]

ionosphere = pd.read_csv(url, header=None, names=column_names)

ionosphere["label"] = ionosphere["label"].map({"b": 0, "g": 1})

In [ ]:
X = ionosphere.drop(columns=['label']).values
y = ionosphere['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

In [ ]:
class MLP(nn.Module):
  def __init__(self, input_dim, output_dim):
    super(MLP, self).__init__()
    self.fc1 = nn.Linear(input_dim, 32)
    self.fc2 = nn.Linear(32, 64)
    self.fc3 = nn.Linear(64, 64)
    self.fc4 = nn.Linear(64, 32)
    self.fc5 = nn.Linear(32, 16)
    self.fc6 = nn.Linear(16, output_dim)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.relu(self.fc4(x))
    x = F.relu(self.fc5(x))
    x = self.fc6(x)
    return x

In [ ]:
# Classe Partícula para PSO adaptado ao treinamento da rede neural
class Particle:
    def __init__(self, model, device):
        self.model = copy.deepcopy(model).to(device)
        self.best_model = copy.deepcopy(model).to(device)
        # self.position = {name: torch.zeros_like(param).to(device) for name, param in model.named_parameters()}
        # self.velocity = {name: torch.zeros_like(param).to(device) for name, param in model.named_parameters()}

        # Definir os limites do espaço de busca e a escala da velocidade
        low = -10.0  # Limite inferior do espaço de busca
        high = 10.0  # Limite superior do espaço de busca
        velocity_scale = 0.1  # Escala para as velocidades iniciais

        # Inicializar a posição com valores aleatórios uniformes no intervalo [low, high]
        self.position = {name: torch.rand_like(param).to(device) * (high - low) + low for name, param in model.named_parameters()}

        # Inicializar a velocidade com valores aleatórios pequenos (normalmente distribuídos)
        self.velocity = {name: torch.randn_like(param).to(device) * velocity_scale for name, param in model.named_parameters()}

        self.best_score = float('inf')
        self.device = device

        # Inicializar o otimizador (por exemplo, Adam)
        #self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0007, weight_decay=1e-5)

    def pso_sgd(self, global_best_model, inertia, c1, c2, learning_rate, beta1, beta2, epsilon, m, v, t):
        for name, param in self.model.named_parameters():
            if param.grad is None:
                continue

            local_rand = random.random()
            global_rand = random.random()

            # Atualização da velocidade
            self.velocity[name] = (
                inertia * self.velocity[name]
                + c1 * local_rand * (self.best_model.state_dict()[name].to(self.device) - param.data)
                + c2 * global_rand * (global_best_model.state_dict()[name].to(self.device) - param.data)
            )

            self.position[name] = param.data + self.velocity[name]
            param.data = self.position[name]

            # Move m[name] e v[name] para o mesmo dispositivo de param
            m[name] = m[name].to(param.device)
            v[name] = v[name].to(param.device)

            # Atualização do Adam
            m[name] = beta1 * m[name] + (1 - beta1) * param.grad
            v[name] = beta2 * v[name] + (1 - beta2) * (param.grad ** 2)

            m_hat = m[name] / (1 - beta1 ** t)
            v_hat = v[name] / (1 - beta2 ** t)

            # param.data = self.position[name] - learning_rate * m_hat / (torch.sqrt(v_hat) + epsilon)
            param.data = self.position[name] - learning_rate * param.grad
            # param.data = self.position[name] + learning_rate * param.grad

    def evaluate_test(self, x_test, y_test, criterion):
        self.model.eval()
        total_loss = 0
        correct = 0
        total = len(x_test)

        with torch.no_grad():

          outputs = self.model(x_test)
          loss = criterion(outputs, y_test)
          total_loss += loss.item()

          _, predicted = torch.max(outputs.data, 1)
          accuracy = (predicted == y_test).sum().item() / total

        avg_loss = total_loss / total
        return avg_loss, accuracy * 100

    def evaluate_train(self, x_train, y_train, criterion):
        """Calcula o erro (perda) e a acurácia da partícula utilizando o otimizador Adam."""
        self.model.train()  # Colocar o modelo em modo de treinamento
        total_loss = 0
        correct = 0
        total = len(x_train)

        # Zerar gradientes acumulados
        self.optimizer.zero_grad()

        # Forward pass
        outputs = self.model(x_train)

        # Cálculo do erro (loss)
        loss = criterion(outputs, y_train)

        # Backward pass (propagação do gradiente)
        loss.backward()

        # Atualizar os pesos usando Adam
        self.optimizer.step()

        # Acumular o erro total
        total_loss += loss.item()

        # Cálculo da acurácia

        _, predicted = torch.max(outputs.data, 1)
        accuracy = (predicted == y_train).sum().item() / total
        # Cálculo da perda média e acurácia
        avg_loss = total_loss / total


        return avg_loss, accuracy * 100

In [ ]:
pop_size = 10
num_epochs = 150
inertia = 0.9
c1, c2 = 0.6, 0.9
learning_rate = 0.0002
beta1, beta2 = 0.5, 0.999
epsilon = 1e-8

In [ ]:
model = MLP(input_dim=34, output_dim=2)
particles = [Particle(model, device) for _ in range(pop_size)]

global_best_model = copy.deepcopy(particles[0].model)
global_best_score = float('inf')

criterion = nn.CrossEntropyLoss()

# Inicializar m e v para Adam
m = {name: torch.zeros_like(param) for name, param in model.named_parameters()}
v = {name: torch.zeros_like(param) for name, param in model.named_parameters()}

# Loop de treinamento do PSO
for epoch in range(num_epochs):
    inertia = 0.9 - ((0.9-0.4)/num_epochs)*epoch
    for particle in particles:
        # Colocar o modelo em modo de treinamento
        particle.model.train()

        particle.optimizer.zero_grad()

        # Treinar a partícula (atualização de posição)
        particle.pso_sgd(global_best_model, inertia, c1, c2, learning_rate, beta1, beta2, epsilon, m, v, epoch + 1)

        # Avaliar a partícula e atualizar o local best
        val_loss, val_accuracy = particle.evaluate_train(X_train, y_train, criterion)

        if val_loss < particle.best_score:
            particle.best_score = val_loss
            particle.best_model = copy.deepcopy(particle.model)

    #Determinar e atualizar o g-best (modelo global)
    best_particle = min(particles, key=lambda p: p.best_score)
    if best_particle.best_score < global_best_score:
        global_best_score = best_particle.best_score
        global_best_model = copy.deepcopy(best_particle.best_model)

    # Avaliar e imprimir a cada época
    if (epoch + 1) % 10 == 0:
        val_loss, val_accuracy = best_particle.evaluate_test(X_test, y_test, criterion)
        print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.2f}, Validation Accuracy: {val_accuracy:.2f}')

Epoch 10/150, Validation Loss: 0.01, Validation Accuracy: 63.21
Epoch 20/150, Validation Loss: 0.01, Validation Accuracy: 63.21
Epoch 30/150, Validation Loss: 0.01, Validation Accuracy: 63.21
Epoch 40/150, Validation Loss: 0.01, Validation Accuracy: 63.21
Epoch 50/150, Validation Loss: 0.00, Validation Accuracy: 75.47
Epoch 60/150, Validation Loss: 0.00, Validation Accuracy: 91.51
Epoch 70/150, Validation Loss: 0.00, Validation Accuracy: 93.40
Epoch 80/150, Validation Loss: 0.00, Validation Accuracy: 95.28
Epoch 90/150, Validation Loss: 0.00, Validation Accuracy: 96.23
Epoch 100/150, Validation Loss: 0.00, Validation Accuracy: 96.23
Epoch 110/150, Validation Loss: 0.00, Validation Accuracy: 96.23
Epoch 120/150, Validation Loss: 0.00, Validation Accuracy: 98.11
Epoch 130/150, Validation Loss: 0.00, Validation Accuracy: 98.11
Epoch 140/150, Validation Loss: 0.00, Validation Accuracy: 98.11
Epoch 150/150, Validation Loss: 0.00, Validation Accuracy: 98.11
